In [1]:
import parmed
import numpy as np
import itertools

from pdbfixer import PDBFixer
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *

from sys import stdout

from scipy.spatial.distance import cdist


you got numpy


# Load inputs:

Loading the protein/bilayer system:

In [2]:
topology.Topology.loadBondDefinitions('../data/dppc.xml')
prot_pdb = PDBFile('../data/charmm-gui/step5_assembly.pdb')
omm_forcefield = app.ForceField('amber14-all.xml', 'amber14/tip3pfb.xml', 'amber14/lipid17.xml')
prot_system = omm_forcefield.createSystem(prot_pdb.topology, rigidWater=False)
prot_structure = parmed.openmm.load_topology(prot_pdb.topology,
                                           prot_system,
                                           xyz=prot_pdb.positions)

lipid_indices = np.array([count for count, i in enumerate(prot_pdb.topology.atoms()) if i.residue.name=='DPPC'])
protein_indices = np.array([count for count, i in enumerate(prot_pdb.topology.atoms()) if i.residue.name not in ["DPPC", "CL", "NA", "HOH"]])

Load parsley-parameterized drug system:

In [3]:
drug_system = XmlSerializer.deserialize(open('drug_system.xml').read())

drug_pdbfile = PDBFile('./cbd.pdb')

drug_structure = parmed.openmm.load_topology(drug_pdbfile.topology,
                                                drug_system,
                                                xyz=drug_pdbfile.positions)

# Add drug molecules, randomly placed

Add drug molecules at random orientations and x,y,z (but keeping within the bilayer):

In [4]:
##first, get the appropriate x,y,z ranges:
protein_system_coordinates = prot_structure.coordinates

#how far off from the box do you want to look for new COMs?
offset = 10

x_range = (np.min(protein_system_coordinates[:,0]), np.max(protein_system_coordinates[:,0])+offset)
y_range = (np.min(protein_system_coordinates[:,1]), np.max(protein_system_coordinates[:,1])+offset)

#only in the z range of the protein, to ensure it's close to the bilayer since CBD is highly hydrophobic. 
z_range = (np.min(protein_system_coordinates[lipid_indices][:,2])+4, 
           np.max(protein_system_coordinates[lipid_indices][:,2])-4)

In [5]:
##generates a rotation matrix using three random angles chosen uniformly in range -np.pi, np.pi
def make_rotation_matrix():
    angles = np.random.uniform(-np.pi, np.pi, (3))
    c1, s1 = np.cos(angles[0]), np.sin(angles[0])
    c2, s2 = np.cos(angles[1]), np.sin(angles[1])
    c3, s3 = np.cos(angles[2]), np.sin(angles[2])
    return np.array(
        [[  c2*c3            , -c2*s3            ,  s2               ],
         [  c1*s3 + c3*s1*s2 ,  c1*c3 - s1*s2*s3 , -c2*s1            ],
         [  s1*s3 - c1*c3*s2 ,  c3*s1 + c1*s2*s3 ,  c1*c2            ]]).T

In [6]:
##how close can the proposed drug molecule be to an existing atom coordinate?
buffer = 3 #this is angstrom here

orig_drug_coords = drug_structure.coordinates

num_drugs = 0
num_wanted = 15

complex_structure = copy.copy(prot_structure)

while num_drugs<num_wanted:
    #make a proposed COM:
    x = np.random.uniform(x_range[0], x_range[1])
    y = np.random.uniform(y_range[0], y_range[1])
    z = np.random.uniform(z_range[0], z_range[1])
    
    #make a random proposed three-axis rotation:
    rot_mat = make_rotation_matrix()
    
    #generate new coordinates for the drug:
    new_coords = np.matmul(orig_drug_coords, rot_mat)+np.array([x,y,z])
    
    #minimum distance to any existing atom:
    mindist = np.min(cdist(new_coords, complex_structure.coordinates))
    #if no overlaps below 'buffer' tolerance, then add the ligand.
    if mindist>buffer:            
        drug_structure.coordinates = new_coords
        complex_structure = complex_structure+drug_structure            
        num_drugs+=1
        print(mindist, x,y,z)

3.5413625248286755 6.830504361654995 42.4858222362644 -18.147306479900806
9.035640202283064 47.75818501849703 -11.672304915179975 -10.149752775957749
7.399039396694399 13.772065560070558 47.3010509119285 -1.5989406498494318
9.365797008627121 -38.64297831204941 46.88719948446143 -2.790766523602784
3.1415971243788094 44.297762538675414 -28.597069578436525 1.4526240575959477
11.494837951673569 49.09352175425764 -13.381346498252377 15.806758639706693
4.3739664301816745 14.739161043563549 47.64620686526682 18.05447180823262
6.929845935919474 -26.327491980856685 47.61548108621922 7.848003829391228
8.392631850721182 47.98876558718945 32.29092381486882 13.162974945102668
5.808650625149637 45.53114193072844 37.94996021142596 -4.6843504921767565
3.954186028317514 49.57094171926653 -29.759493145182816 12.624912293456205
3.207801640770654 37.8882943581107 38.073475296999845 3.1202059594602183
7.122743003697673 -10.176730453727934 49.146004796687734 11.665332547744079
4.302379126423787 43.583312116

set periodic boundary conditions,
then shift so that protein COM is at (0,0,0)

In [8]:
#first, shift so that minimum coordinate is at (0,0,0). This allows to set PBC
complex_structure.coordinates = complex_structure.coordinates-np.min(complex_structure.coordinates, axis=0)
#set PBC with a short buffer:
newbox = (np.max(complex_structure.coordinates,axis=0)+np.array(2))*angstroms
complex_structure.box = (newbox[0], newbox[0], newbox[2], 90, 90, 90)

#now return coordinates so that protein COM is at (0,0,0):
complex_structure.coordinates = complex_structure.coordinates-np.mean(complex_structure.coordinates[protein_indices], axis=0)

In [9]:
##Save a PDB just so you can visualize the output as a sanity check:
complex_structure.save('./test.pdb', overwrite=True)

# Simulation setup:

In [10]:
#turn into a System object
complex_system = complex_structure.createSystem(nonbondedMethod=PME,
                                                nonbondedCutoff=0.9*nanometer,
                                               constraints=HBonds,
                                                rigidWater=True)

Collect a set of lists storing the atom indices for each drug residue:

In [11]:
resis = list()
indices = list()
for count, a in enumerate(complex_structure.topology.atoms()):
    if a.residue.name=='UNL':
        resis.append(a.residue.index)
        indices.append(count)
        
drugs = list()
unique_resis=list(set(resis))
for r in unique_resis:
    drugs.append([k for j,k in zip(resis,indices) if j==r])


This cell adds a repulsive bias between each pair of drug molecules. This was inspired by the bias used in the SILCS fragment-screening approach. 

In [12]:
drugForce = CustomNonbondedForce("drugForceMultiplier*exp(-12.5*r)")
drugForce.addGlobalParameter("drugForceMultiplier", 0.01*kilocalories_per_mole/angstroms**2)
for particle in range(complex_system.getNumParticles()):
    drugForce.addParticle()
groups = [i for i in itertools.combinations(drugs,2)]
for group in groups:
    drugForce.addInteractionGroup(group[0], group[1])
complex_system.addForce(drugForce)


5

In [13]:
##add a barostat. Not only will this initially fix the big vacuums between cells,
##it will be maintained throughout to keep pressure reasonable. 
barostat = MonteCarloMembraneBarostat(1*bar, 200*bar*nanometer, 310*kelvin, MonteCarloMembraneBarostat.XYIsotropic, MonteCarloMembraneBarostat.ZFree)
complex_system.addForce(barostat)

6

In [14]:
#This is the final force to add during the equil phase. 
#This keeps all the protein atoms restrained. It is periodic, so if the protein
#COM happens to hop across the cell, the restraint centres also move.
#It is added last so it can be removed easily later. 
proteinForce = CustomExternalForce("k1*periodicdistance(x, y, z, x0, y0, z0)^2")
proteinForce.addGlobalParameter("k1", 10*kilocalories_per_mole/angstroms**2)
proteinForce.addPerParticleParameter("x0")
proteinForce.addPerParticleParameter("y0")
proteinForce.addPerParticleParameter("z0")
for count, coord in enumerate(complex_structure.positions):
    if count in protein_indices:
        proteinForce.addParticle(int(count), [coord[0], coord[1], coord[2]])        

#for count, coord in enumerate(complex_structure.coordinates[protein_indices]):
#    proteinForce.addParticle(int(count), [coord[0], coord[1], coord[2]])        
print("Does it use periodic conditions?", proteinForce.usesPeriodicBoundaryConditions())
complex_system.addForce(proteinForce)


Does it use periodic conditions? True


7

# Equilibrate:

In [15]:
integrator = LangevinIntegrator(310*kelvin, 1/picosecond, 0.002*picoseconds)
platform = Platform.getPlatformByName('CUDA')
prop = {'CudaPrecision':'single', 'CudaDeviceIndex':'0'}
simulation = Simulation(complex_structure.topology, complex_system, integrator, platform, prop)

In [16]:
simulation.context.setPositions(complex_structure.positions)
simulation.minimizeEnergy(maxIterations=150)

In [17]:
##PSF files don't like having numbered atom types, because at 10,000 VMD fails
for a in complex_structure.atoms:
    a.type = '0'
complex_structure.save('output.psf', overwrite=True)
complex_structure.save('output.pdb', overwrite=True)


In [18]:
simulation.reporters.append(DCDReporter('output.dcd', 20000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True, potentialEnergy=True, temperature=True, speed=True))

In [ ]:
simulation.context.setParameter('k1', max(0, float(10-0)))
one_ns = 500000
for i in range(10):
     simulation.step(one_ns*5)
     simulation.context.setParameter('k1', max(0, float(10-i)))



with open('solvated_equilibrated_system.xml', 'w') as f:
    f.write(
            XmlSerializer.serialize(
                complex_system
            )
    )
    
with open('solvated_equilibrated_state.xml', 'w') as f:
    f.write(
            XmlSerializer.serialize(
                simulation.context.getState(getPositions=True,
                                     getForces=True, getEnergy=True,
                                     enforcePeriodicBox=True)
            )
    )

#"Step","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
1000,-522065.6201141202,282.2518654053632,0
2000,-515225.6305047795,306.88191044739375,156
3000,-514688.6393923373,310.5103766066198,155
4000,-517314.10865908023,310.7477138674237,155
5000,-518005.21717048204,310.8529027996792,155
6000,-521469.1539585418,311.54368362941347,155
7000,-522230.3920858111,310.8012326554031,155
8000,-520107.8934050696,307.9860631061658,155
9000,-523824.8153256234,308.50714149486043,155
10000,-522967.5535314069,310.9995326096296,155
11000,-523334.6731689647,308.7097576048627,155
12000,-524460.6502603353,310.36265051886846,155
13000,-523612.97453947365,308.52189206865165,154
14000,-525609.6009320258,310.2925784550429,154
15000,-525198.8112378134,309.73969971175734,154
16000,-525216.2465198957,312.2648350043302,154
17000,-526702.7110404484,311.895283224311,154
18000,-525796.3613721528,311.56511901789867,154
19000,-524348.9546751818,310.0642462743141,154
20000,-524214.5843069367,311.07670032

172000,-548415.705083529,312.3930885321886,146
173000,-548787.2918125121,310.1306112710565,146
174000,-549754.7375441627,308.85870980386306,146
175000,-548872.9500890388,309.11810505834296,146
176000,-550093.2685983512,310.3580319546561,146
177000,-549357.8334607407,309.67241121823844,146
178000,-548836.8285957277,309.41393068450367,146
179000,-548107.6390572898,308.3433729235511,146
180000,-548608.805389455,308.73929652969883,146
181000,-548426.4193317466,309.2821178703331,146
182000,-547474.37608427,308.6957866140324,146
183000,-547399.210414092,311.0649024494906,146
184000,-548104.6799437073,310.71218329126134,146
185000,-548817.360692835,308.4632354332653,146
186000,-550071.7119578877,307.82530269502956,146
187000,-548861.8551226044,308.0808512781292,146
188000,-549611.3565528337,310.6365630365647,146
189000,-548576.2104435177,310.81962424699725,146
190000,-547861.935692077,309.73622605304075,146
191000,-549397.7007009387,308.8000837374302,146
192000,-549634.2442136053,310.67509865

343000,-550277.6570553659,308.8895942834404,144
344000,-549550.0506846062,307.79009590140464,144
345000,-549783.2048836462,309.7345272488337,144
346000,-551136.548828179,312.0280138108476,144
347000,-550398.6822112761,311.0140059399634,144
348000,-551014.8720504837,310.7306086517881,144
349000,-549297.7823034422,308.56817142077097,144
350000,-552361.1585966423,309.61453763032927,144
351000,-550862.1348767681,311.3750428488874,144
352000,-549072.1105671343,310.64930978685913,144
353000,-548549.7507487121,308.6899051205033,144
354000,-551277.3081478891,310.56326991517864,144
355000,-550000.3661254905,308.3549170647016,144
356000,-550364.6375078456,309.8252518310411,144
357000,-550197.2712073782,312.4688488211561,144
358000,-548968.2613615002,310.56395402694363,144
359000,-551393.2041761065,311.552367222182,144
360000,-551050.5618868168,311.67772959653615,144
361000,-550453.4289259352,309.16703611318655,144
362000,-550571.390409342,309.88301119713384,144
363000,-550532.1852182834,310.1984

513000,-550418.1654718365,309.36547225367815,143
514000,-551299.6200472866,310.90712053560753,143
515000,-551438.3122100853,310.83533764858663,143
516000,-550662.451372256,313.03973769707903,143
517000,-550700.1928013782,311.5057012804083,143
518000,-551737.3432202083,309.9404808971223,143
519000,-551430.6059232075,309.20824464308237,143
520000,-551347.868593771,308.719655373551,143
521000,-552224.9009124897,310.4701193169818,143
522000,-550733.3881556536,311.1861439659219,143
523000,-550771.7098623859,308.52910047858643,143
524000,-551114.5902026356,308.6336926225687,143
525000,-550541.1115176613,311.46833070839836,143
526000,-550999.480471075,311.5036319740018,143
527000,-551120.7630028492,311.18777003922327,143
528000,-550252.1050931946,309.848385243692,143
529000,-548340.8425070886,309.36144112196945,143
530000,-550343.7777423486,307.0939401718028,143
531000,-551529.5478165825,311.1603840370882,143
532000,-550999.2737183259,311.66134245951093,143
533000,-550874.0135658756,309.02958

683000,-549187.88075419,309.7942975559774,143
684000,-550144.154701313,310.25389272307785,143
685000,-550297.5648733708,312.2828539994184,143
686000,-551274.2955179974,309.46097870374626,143
687000,-550270.0417808127,310.71292106062816,143
688000,-550990.0262151156,307.40332381609824,143
689000,-551350.3738840218,311.29218476312565,143
690000,-550887.41662734,312.8011495342099,143
691000,-551037.6315828431,308.64051887623486,143
692000,-550969.6089983322,310.30046223492985,143
693000,-551281.8347868868,310.2627969796043,143
694000,-550652.4369451045,309.4511443977418,143
695000,-550737.1556960223,310.3710728269901,143
696000,-550969.5247339825,310.3485295381944,143
697000,-550761.7693671025,310.39042116712056,143
698000,-551005.9419077132,309.83225978507124,143
699000,-551816.4406438386,308.39834447964034,143
700000,-551301.4491641237,311.29682235065263,143
701000,-551104.2108856775,309.55120036874786,143
702000,-552080.2592176879,309.36133163944476,143
703000,-551711.6217071326,310.02

853000,-551051.3104099194,308.9889792363928,143
854000,-550380.8029984958,311.4147942378906,143
855000,-551311.7432916793,310.7446419495988,143
856000,-551228.9984890437,311.8169041823739,143
857000,-551912.2265103101,311.24397389518555,143
858000,-551214.8329092585,310.3608073960527,143
859000,-551518.7146051917,312.52621153187664,143
860000,-551232.1614397131,311.64051397945065,143
861000,-551032.6296218904,310.846796901746,143
862000,-552005.6198473461,308.093643585851,143
863000,-552303.1158894887,308.73741609977685,143
864000,-551470.7768784007,310.7939084906159,143
865000,-552556.0543104992,310.13571008931575,143
866000,-550077.4251378584,310.2177990267138,143
867000,-551333.9627597099,312.99092641922726,143
868000,-551405.0375799756,309.73577513147865,143
869000,-552954.1296737273,310.03583214091975,143
870000,-552631.755666744,310.0483554253053,143
871000,-552006.1997856004,310.1891901803346,143
872000,-551965.5159523874,310.7839478192982,143
873000,-551021.2596067213,309.87396

1023000,-550108.749433496,311.098542674481,142
1024000,-551238.690736555,309.6800782347958,142
1025000,-550162.493289791,310.27042116233713,142
1026000,-550573.2776713213,310.435219679826,142
1027000,-551516.2226717947,310.0558171501444,142
1028000,-550425.4147488093,310.82843873775437,142
1029000,-551060.5644491073,310.85033634972103,142
1030000,-551301.6087735314,308.60364152944186,142
1031000,-550993.9575937949,311.63065539020687,142
1032000,-551968.9579993146,308.9188852138396,142
1033000,-551095.608123207,311.1044579419914,142
1034000,-552093.4654844785,311.1899357180041,142
1035000,-552065.8184153507,309.65084296195283,142
1036000,-552689.712668315,309.51653993300744,142
1037000,-552586.4040380768,310.5589734542677,142
1038000,-553006.7118922309,308.6004023210117,142
1039000,-553124.6424167254,309.2491170868881,142
1040000,-551836.9405574324,307.1726143323889,142
1041000,-551422.107942964,309.95899394092925,142
1042000,-549521.1066040243,306.89642793981255,142
1043000,-552468.633

1190000,-552127.261076556,306.75184265871195,142
1191000,-552498.7546970528,310.0078056097701,142
1192000,-552885.6049547018,312.1246653002891,142
1193000,-552114.8277949104,307.88135738897245,142
1194000,-551663.0247970573,312.6189367979319,142
1195000,-551326.8497063229,310.4040338576328,142
1196000,-551008.3612620402,310.11845988042086,142
1197000,-550817.3315669307,310.0366155343623,142
1198000,-551311.0535050547,310.54237614987727,142
1199000,-551753.1219984139,309.8774666686413,142
1200000,-551441.2793164258,309.75750223600664,142
1201000,-551825.964764379,308.3950172565269,142
1202000,-551585.3703226163,309.60147496574587,142
1203000,-550908.849969849,310.5371053235906,142
1204000,-551837.8789353827,311.1240921808345,142
1205000,-550272.0937118409,311.7414710742644,142
1206000,-552049.6633748254,310.048492388226,142
1207000,-551319.9037267794,311.7441756488581,142
1208000,-550639.9584965585,308.13617597654076,142
1209000,-551444.6356778541,309.1005953853972,142
1210000,-551605.4

1357000,-552814.7238379754,310.39081531498556,142
1358000,-553009.8324840339,310.01110259215415,142
1359000,-554360.3003665679,308.9866039881988,142
1360000,-551269.9805863374,308.43186429249164,142
1361000,-551649.1827799072,307.7791007991885,142
1362000,-553097.3176707276,310.3923931849548,142
1363000,-552069.134299458,309.52755295212785,142
1364000,-552594.2086754004,311.2339452037791,142
1365000,-552591.7428068775,310.72863089034445,142
1366000,-552897.2778752777,310.83366527180414,142
1367000,-552621.1050050044,311.2590120534096,142
1368000,-550686.0812218427,311.58084918628515,142
1369000,-551573.0487146792,310.3439239753143,142
1370000,-552127.348228063,307.9198324898242,142
1371000,-551256.8270644802,310.52005673923696,142
1372000,-551645.0298779272,309.7265591158044,142
1373000,-551746.7878179704,307.7622847579924,142
1374000,-552440.6748783365,310.05202375102766,142
1375000,-551517.6237473781,309.45098338769986,142
1376000,-551140.1142769549,311.8017297561935,142
1377000,-551

1524000,-551939.8940034937,311.2067148273224,142
1525000,-551852.01928504,309.2053294617231,142
1526000,-551639.6797332489,309.5520732211905,142
1527000,-552429.3850851888,309.22259572457574,142
1528000,-551351.9952909616,308.56171169898187,142
1529000,-550812.6531122094,308.01243921221663,142
1530000,-551474.5169918262,311.1620719127971,142
1531000,-551106.7357373545,310.4692558464512,142
1532000,-552491.3019779078,310.4196425794942,142
1533000,-551682.9828760047,308.1810623884626,142
1534000,-553114.0358810881,308.43010725665056,142
1535000,-552382.2866724199,307.0798218224887,142
1536000,-551135.537845037,308.74086983665427,142
1537000,-551166.0458949357,311.68249301808476,142
1538000,-551082.310380077,310.8791274971725,142
1539000,-551278.1394992396,309.7767417130307,142
1540000,-551685.416583274,308.1786170085433,142
1541000,-552354.7778802384,309.49424352618905,142
1542000,-552202.4941960643,311.9050864494271,142
1543000,-551751.1105249431,310.6875144341607,142
1544000,-551336.31